In [3]:
%pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 12.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 9.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 11.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 10.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 12.3 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [4]:
import seaborn as sns
import matplotlib as plt
import pandas as pd
import numpy as np
import plotly.express as px

from datasets import load_dataset

from sklearn.model_selection import train_test_split,cross_val_score
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import confusion_matrix, accuracy_score, f1_score, recall_score, roc_curve, roc_auc_score

In [5]:
# this cell may take up to 30 seconds to run to convert the Hugginface Dataset class to Pandas DataFrame for better EDA

HF_SPOTIFY_DATASET_PATH = "maharshipandya/spotify-tracks-dataset"
# https://huggingface.co/datasets/maharshipandya/spotify-tracks-dataset

ds = load_dataset(HF_SPOTIFY_DATASET_PATH) #only has 'train' split as key
df = pd.DataFrame(ds["train"])
df.head()

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/4.68k [00:00<?, ?B/s]

dataset.csv:   0%|          | 0.00/20.1M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/114000 [00:00<?, ? examples/s]

,Unnamed: 0,track_id,artists,album_name,track_name,popularity,duration_ms,explicit,danceability,energy,...,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,time_signature,track_genre
0,0,5SuOikwiRyPMVoIQDJUgSV,Gen Hoshino,Comedy,Comedy,73,230666,False,0.676,0.4610,...,-6.746,0,0.1430,0.0322,0.000001,0.3580,0.715,87.917,4,acoustic
1,1,4qPNDBW1i3p13qLCt0Ki3A,Ben Woodward,Ghost (Acoustic),Ghost - Acoustic,55,149610,False,0.420,0.1660,...,-17.235,1,0.0763,0.9240,0.000006,0.1010,0.267,77.489,4,acoustic
2,2,1iJBSr7s7jYXzM8EGcbK5b,Ingrid Michaelson;ZAYN,To Begin Again,To Begin Again,57,210826,False,0.438,0.3590,...,-9.734,1,0.0557,0.2100,0.000000,0.1170,0.120,76.332,4,acoustic
3,3,6lfxq3CG4xtTiEg7opyCyx,Kina Grannis,Crazy Rich Asians (Original Motion Picture Sou...,Can't Help Falling In Love,71,201933,False,0.266,0.0596,...,-18.515,1,0.0363,0.9050,0.000071,0.1320,0.143,181.740,3,acoustic
4,4,5vjLSffimiIP26QG5WcN2K,Chord Overstreet,Hold On,Hold On,82,198853,False,0.618,0.4430,...,-9.681,1,0.0526,0.4690,0.000000,0.0829,0.167,119.949,4,acoustic


In [6]:
# show all columns with only 2 values
df[[col for col in df.columns if len(df[col].unique()) == 2]]

,explicit,mode
0,False,0
1,False,1
2,False,1
3,False,1
4,False,1
...,...,...
113995,False,1
113996,False,0
113997,False,0
113998,False,1


In [7]:
df.dropna(inplace=True)

In [8]:
print((df['explicit']==0).sum(), (df['explicit']==1).sum())


104252 9747


# 1) selected binary categorical response: explicit
using KNN

In [9]:
# Setting up the response variable 'explicit' for binary classification.
y = df['explicit']

# Selecting features 'speechiness', 'danceability', and 'instrumentalness' based on their high correlation with 'explicit'.
# highest absolute correlation with explicit (> |0.1|)
X = df[['speechiness', 'danceability', 'instrumentalness']]

x, X_test, Y, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

X_train, X_val, y_train, y_val = train_test_split(x, Y, test_size=0.25, random_state=42)


In [10]:
# Checking the shapes of training, testing, and validation sets to confirm the data split.
print(X_train.shape)
print(X_test.shape)
print(X_val.shape)
print(y_train.shape)
print(y_test.shape)
print(y_val.shape)

(68399, 3)
(22800, 3)
(22800, 3)
(68399,)
(22800,)
(22800,)


In [11]:
# Initializing and training the K-Nearest Neighbors (KNN) model on the training data.
knn = KNeighborsClassifier()
knn.fit(X_train, y_train)

KNeighborsClassifier()

In [12]:
# Testing different values of k (from 1 to 20) for KNN and recording the cross-validation score for each to find the optimal k.
k_values = [i for i in range (1,21)]
scores = []

for k in k_values:
    knn = KNeighborsClassifier(n_neighbors=k)
    score = cross_val_score(knn, X_train, y_train, cv=5)
    scores.append(np.mean(score))
    print("K:", k, "score:", np.mean(score))

K: 1 score: 0.902966465008685
K: 2 score: 0.9195310137321518
K: 3 score: 0.9088291043726479
K: 4 score: 0.9169139809217954
K: 5 score: 0.9121186005013021
K: 6 score: 0.9168408569245811
K: 7 score: 0.9140922986060076
K: 8 score: 0.9170455588003807
K: 9 score: 0.9151157032442695
K: 10 score: 0.9167824062495592
K: 11 score: 0.9151741881203485
K: 12 score: 0.9158467166344109
K: 13 score: 0.9149256511774355
K: 14 score: 0.9158759574692757
K: 15 score: 0.9151449579733137
K: 16 score: 0.9157005252854826
K: 17 score: 0.9150718628332412
K: 18 score: 0.9157736279070363
K: 19 score: 0.9150718692459392
K: 20 score: 0.9158028698106844


In [13]:
# Training the KNN model with the optimal number of neighbors (k=4) determined from cross-validation.
knn = KNeighborsClassifier(n_neighbors = 4)
knn.fit(X_train, y_train)

KNeighborsClassifier(n_neighbors=4)

Best K value is 4. 2 has a higher score but is too small and is likely overfitting the data set. 4 is a safer bet

In [14]:
# Generating probability predictions and class predictions on the training set using the trained KNN model.
y_proba = knn.predict_proba(X_train)
y_pred = knn.predict(X_train)

# 2) evaluation metrics

###Training Set Metrics

confusion matrix

In [15]:
cm = confusion_matrix(y_train, y_pred)

#pandas dataframe
pd.DataFrame(
cm,
columns=['Predicted: Not Explicit', 'Predicted: Explicit'],
index=['Actual: Not Explicit', 'Actual: Explicit']
)

,Predicted: Not Explicit,Predicted: Explicit
Actual: Not Explicit,62059,389
Actual: Explicit,3963,1988


prediction accuracy and error

In [16]:
accuracy = accuracy_score(y_train, y_pred)
prediction_error = 1 - accuracy
print("Accuracy:", accuracy)
print("Prediction Error", prediction_error)

Accuracy: 0.9363733387915028
Prediction Error 0.06362666120849725


TPR, FPR, F1 SCORE

In [17]:
true_positive_rate = recall_score(y_train, y_pred)
true_negative_rate = cm[0, 0] / (cm[0, 0] + cm[0, 1])
f1 = f1_score(y_train, y_pred)

print("TPR:", true_positive_rate)
print("FPR:", true_negative_rate)
print("F1:", f1)

TPR: 0.3340615022685263
FPR: 0.9937708173200103
F1: 0.47742555235350626


### Validation Set Metrics

In [18]:
y_val_proba = knn.predict_proba(X_val)
y_val_pred = knn.predict(X_val)

cm_val = confusion_matrix(y_val, y_val_pred)

#pandas dataframe
pd.DataFrame(
cm_val,
columns=['Predicted: Not Explicit', 'Predicted: Explicit'],
index=['Actual: Not Explicit', 'Actual: Explicit']
)

,Predicted: Not Explicit,Predicted: Explicit
Actual: Not Explicit,20606,294
Actual: Explicit,1455,445


In [19]:
val_accuracy = accuracy_score(y_val, y_val_pred)
val_prediction_error = 1 - val_accuracy
print("Accuracy:", val_accuracy)
print("Prediction Error", val_prediction_error)

Accuracy: 0.9232894736842105
Prediction Error 0.07671052631578945


In [20]:
val_true_positive_rate = recall_score(y_val, y_val_pred)
val_true_negative_rate = cm_val[0, 0] / (cm_val[0, 0] + cm_val[0, 1])
val_f1 = f1_score(y_val, y_val_pred)

print("TPR:", val_true_positive_rate)
print("FPR:", val_true_negative_rate)
print("F1:", val_f1)

TPR: 0.23421052631578948
FPR: 0.985933014354067
F1: 0.3372489579386131


# 3) ROC, AUC

In [ ]:
# Calculating the ROC curve values (false positive rate, true positive rate, thresholds) for the KNN model on the validation set and plotting it.

knn_fpr_sample, knn_tpr_sample, knn_thresholds_sample = roc_curve(y_val, y_val_proba[:,1])

roc_knn_sample = pd.DataFrame({
    'False Positive Rate': knn_fpr_sample,
    'True Positive Rate': knn_tpr_sample,
    'Model': 'KNN'
}, index=knn_thresholds_sample)

roc_sample_df = pd.concat([roc_knn_sample])

px.line(roc_sample_df, y='True Positive Rate', x='False Positive Rate',
        color='Model',
        width=700, height=500
)

In [21]:
# AUC for the KNN model on the validation set.

knn_auc_sample = roc_auc_score(y_val, y_val_proba[:,1])
print('KNN AUC:', knn_auc_sample.round(3))

KNN AUC: 0.777


In [ ]:
# 5-fold cross-validation on the validation set to calculate and print AUC and accuracy scores for each fold.

from sklearn.model_selection import cross_val_score
num_folds = 5

auc_values = cross_val_score(knn, X_val, y_val, cv=num_folds, scoring='roc_auc')
accuracy_values = cross_val_score(knn, X_val, y_val, cv=num_folds, scoring='accuracy')


for i in range(num_folds):
  print(f"Crossfold {i + 1}:\nAUC: {auc_values[i]}\nAccuracy: {accuracy_values[i]}\n\n")

Crossfold 1:
AUC: 0.7170202719717955
Accuracy: 0.9138157894736842


Crossfold 2:
AUC: 0.709592986653236
Accuracy: 0.9184210526315789


Crossfold 3:
AUC: 0.6955256862251322
Accuracy: 0.9153508771929825


Crossfold 4:
AUC: 0.7327527700831025
Accuracy: 0.9184210526315789


Crossfold 5:
AUC: 0.7091891211281793
Accuracy: 0.9162280701754386


